# Read input parameters

In [271]:
include("input.jl");

# Function to get atomic radii

In [290]:
𝑟ₐₜₒₘ ;
atomicradius = Dict(
    "H" => 1.2,
    "B" => 1.92,
    "C" => 1.7, 
    "N" => 1.55,
    "O" => 1.52,
    "F" => 1.47
)
atomicradius["H"]

UndefVarError: UndefVarError: 𝑟ₐₜ not defined

# Function to get solvent parameters

In [257]:
# returns a 5-element tuple of solvent properties: 
# (dielectric constant, density, molar mass, number of valence electrons, molecular radius (Ang.) )
function solventparameters(solvent)
    if solvent == "cyclohexane"
        return (2.0165, 0.7781, 84.1595, 36, 2.815)
    elseif solvent == "benzene"
        return (2.2706, 0.8756, 78.1118, 30, 2.63)
    elseif solvent == "argon"
        return (1.43, 1.3954, 39.948, 8, 1.705)
    else 
        println("Solvent not implemented. Try cyclohexane, benzene, or argon")
    end
end

solventparameters (generic function with 1 method)

In [260]:
solventparameters(solvent)

(2.0165, 0.7781, 84.1595, 36, 2.815)

# Function to write Gaussian input files

In [219]:
# function to write Gaussian input files
function write_gaussian_input(geometry,scaling,jobtype)
    # Split the long string geometry into short stings, 
    # and stored in an array called structure
    structure = split(geometry, r"\n\n+(?!$)")
    
    #geometry_new = read(`awk '{print $2, $3, $4}' tmp`, String)
    #structure_new = split(read(pipeline(`echo $geometry`, `awk '{print $2, $3, $4}'`), String), r"\n\n+(?!$)")
    
    #for i in 1:number_of_atoms
     #   structure_array = split(structure_string[i], r"\n+")
    
    #coordinate_1 = split(structure[1], r"\s+")
    #for i in 2:length(coordinate_1)
    
    # (dielectric constant, density, molar mass, 
    #  number of valence electrons, molecular radius)
    solv_para = solventparameters(solvent)
    
    if jobtype == "Vc"
        for i in 1:length(structure)
            for j in 1:length(scaling)
            open("file-$i-Vc.gjf", "w") do f  
                write(f, """
                %chk=file-$i.chk
                %nproc=$nproc
                %mem=$mem
                #p $method $basis 
                # scrf=(iefpcm,solvent=$solvent,read) nosym guess=only pop=none

                title

                $charge $multiplicity

                $(structure[i])

                qrep pcmdoc geomview nodis nocav g03defaults tsare=$tesserae
                nsfe=$number_of_atoms
                eps=$(solv_para[1]) rhos=$(solv_para[2]) solvmw=$(solv_para[3])
                nvesolv=$(solv_para[4]) rsolv=$(solv_para[5])

                $(read(pipeline(`echo $(structure[i])`, `awk -v var=$(solv_para[5]) '{print $2, $3, $4, var, 1.2}'`), String)) $(solv_para[5]) 1.2
                
                """)
            end
            end
        end
    end    
end

write_gaussian_input (generic function with 1 method)

In [220]:
write_gaussian_input(geometry,"Vc")

run(`cat file-1-Vc.gjf`)
#run(`cat file-2-Vc.gjf`)

%chk=file-1.chk
%nproc=8
%mem=1gb
#p wb97xd def2tzvp 
# scrf=(iefpcm,solvent=cyclohexane,read) nosym guess=only pop=none

title

0 1

C    1.10712900   -1.48465400    0.00000000
C   -0.00001100   -0.72877000    0.00000000
C    0.00000000    0.72875800    0.00000000
C   -1.10712300    1.48466300    0.00000000
H    2.09998900   -1.03986800    0.00000000
H    1.05995700   -2.56940600    0.00000000
H   -0.97811200   -1.21104800    0.00000000
H    0.97811000    1.21101800    0.00000000
H   -2.09999700    1.03991100    0.00000000
H   -1.05991600    2.56941400    0.00000000

qrep pcmdoc geomview nodis nocav g03defaults tsare=0.075
nsfe=10
eps=2.0165 rhos=0.7781 solvmw=84.1595
nvesolv=36 rsolv=2.815

1.10712900 -1.48465400 0.00000000 2.815 1.2
-0.00001100 -0.72877000 0.00000000 2.815 1.2
0.00000000 0.72875800 0.00000000 2.815 1.2
-1.10712300 1.48466300 0.00000000 2.815 1.2
2.09998900 -1.03986800 0.00000000 2.815 1.2
1.05995700 -2.56940600 0.00000000 2.815 1.2
-0.97811200 -1.21104800 0.00000000

Process(`cat file-1-Vc.gjf`, ProcessExited(0))

# testing

In [40]:
a=solventparameters("argon")
a[4]

8

In [84]:
structure = split(geometry, r"\n\n+(?!$)")
coordinate = []
for i in structure
    println(i, "\n")
    coordinate = split(structure[i], r"\s+")
end
println(coordinate[1])

C    1.10712900   -1.48465400    0.00000000
C   -0.00001100   -0.72877000    0.00000000
C    0.00000000    0.72875800    0.00000000
C   -1.10712300    1.48466300    0.00000000
H    2.09998900   -1.03986800    0.00000000
H    1.05995700   -2.56940600    0.00000000
H   -0.97811200   -1.21104800    0.00000000
H    0.97811000    1.21101800    0.00000000
H   -2.09999700    1.03991100    0.00000000
H   -1.05991600    2.56941400    0.00000000



ArgumentError: ArgumentError: invalid index: "C    1.10712900   -1.48465400    0.00000000\nC   -0.00001100   -0.72877000    0.00000000\nC    0.00000000    0.72875800    0.00000000\nC   -1.10712300    1.48466300    0.00000000\nH    2.09998900   -1.03986800    0.00000000\nH    1.05995700   -2.56940600    0.00000000\nH   -0.97811200   -1.21104800    0.00000000\nH    0.97811000    1.21101800    0.00000000\nH   -2.09999700    1.03991100    0.00000000\nH   -1.05991600    2.56941400    0.00000000" of type SubString{String}

In [283]:
length(scaling)

7

In [83]:
array = [1;2;3]

3-element Array{Int64,1}:
 1
 2
 3

In [177]:
structure_string = split(strip(geometry, '\n'), r"\n\n+(?!$)")
for i in 1:length(structure_string)
    if i == 1
        structure_array = split(structure_string[i], r"\n+")
    else
        structure_array = hcat(structure_array, split(structure_string[i], r"\n+"))
    end
end
#coordinate = split(structure_array[1], r"\s+")
structure_array

10×3 Array{SubString{String},2}:
 "C    1.10712900   -1.48465400    0.00000000"  …  "C    1.10712900   -1.48465400    0.00000000"
 "C   -0.00001100   -0.72877000    0.00000000"     "C   -0.00001100   -0.72877000    0.00000000"
 "C    0.00000000    0.72875800    0.00000000"     "C    0.00000000    0.72875800    0.00000000"
 "C   -1.10712300    1.48466300    0.00000000"     "C   -1.10712300    1.48466300    0.00000000"
 "H    2.09998900   -1.03986800    0.00000000"     "H    2.09998900   -1.03986800    0.00000000"
 "H    1.05995700   -2.56940600    0.00000000"  …  "H    1.05995700   -2.56940600    0.00000000"
 "H   -0.97811200   -1.21104800    0.00000000"     "H   -0.97811200   -1.21104800    0.00000000"
 "H    0.97811000    1.21101800    0.00000000"     "H    0.97811000    1.21101800    0.00000000"
 "H   -2.09999700    1.03991100    0.00000000"     "H   -2.09999700    1.03991100    0.00000000"
 "H   -1.05991600    2.56941400    0.00000000"     "H   -1.05991600    2.56941400    3.0000000

In [172]:
structure_string = split(geometry, r"\n\n+(?!$)")
a1 = split(structure_string[1], r"\n+")
a2 = split(structure_string[2], r"\n+")
hcat(a1,split(structure_string[2], r"\n+"))

10×2 Array{SubString{String},2}:
 "C    1.10712900   -1.48465400    0.00000000"  …  "C    1.10712900   -1.48465400    0.00000000"
 "C   -0.00001100   -0.72877000    0.00000000"     "C   -0.00001100   -0.72877000    0.00000000"
 "C    0.00000000    0.72875800    0.00000000"     "C    0.00000000    0.72875800    0.00000000"
 "C   -1.10712300    1.48466300    0.00000000"     "C   -1.10712300    1.48466300    0.00000000"
 "H    2.09998900   -1.03986800    0.00000000"     "H    2.09998900   -1.03986800    0.00000000"
 "H    1.05995700   -2.56940600    0.00000000"  …  "H    1.05995700   -2.56940600    0.00000000"
 "H   -0.97811200   -1.21104800    0.00000000"     "H   -0.97811200   -1.21104800    0.00000000"
 "H    0.97811000    1.21101800    0.00000000"     "H    0.97811000    1.21101800    0.00000000"
 "H   -2.09999700    1.03991100    0.00000000"     "H   -2.09999700    1.03991100    0.00000000"
 "H   -1.05991600    2.56941400    0.00000000"     "H   -1.05991600    2.56941400    2.0000000

In [127]:
a="Ch    1 2 3"
replace(a, r"^\s*\S+\s+" => "")

"1 2 3"

In [130]:
replace.(structure_array, r"^\s*\S+\s+" => "")

10-element Array{String,1}:
 "1.10712900   -1.48465400    0.00000000"
 "-0.00001100   -0.72877000    0.00000000"
 "0.00000000    0.72875800    0.00000000"
 "-1.10712300    1.48466300    0.00000000"
 "2.09998900   -1.03986800    0.00000000"
 "1.05995700   -2.56940600    0.00000000"
 "-0.97811200   -1.21104800    0.00000000"
 "0.97811000    1.21101800    0.00000000"
 "-2.09999700    1.03991100    0.00000000"
 "-1.05991600    2.56941400    0.00000000"

In [138]:
A = rand(UInt8, 2,2)
B = rand(UInt8, 2,2)

2×2 Array{UInt8,2}:
 0x6e  0x49
 0x82  0x48

In [139]:
C = [A B]

2×4 Array{UInt8,2}:
 0x58  0xf0  0x6e  0x49
 0x98  0xd5  0x82  0x48

In [140]:
D = [A ; B]

4×2 Array{UInt8,2}:
 0x58  0xf0
 0x98  0xd5
 0x6e  0x49
 0x82  0x48

In [281]:
solv_para = solventparameters(solvent)
open("tmp", "w") do f  
    write(f, "%chk=file-","i",".chk\n")
    write(f, "%nproc=","$nproc","\n")
    write(f, "%mem=",mem,"\n")
    write(f, "#p ",method," ",basis,"\n")
    write(f, "scrf=(iefpcm,solvent=",solvent,",read) nosym guess=only pop=none\n\n")
    write(f, "title\n\n")
    write(f, "$charge"," ", "$multiplicity","\n\n")
    write(f, "$(structure[1])\n\n")
    write(f, "qrep pcmdoc geomview nodis nocav g03defaults tsare=","$tesserae","\n")
    write(f, "nsfe=","$number_of_atoms","\n")
    write(f, "eps=","$(solv_para[1])"," rhos=","$(solv_para[2])"," solvmw=","$(solv_para[3])","\n")
end


 #               nvesolv=$(solv_para[4]) rsolv=$(solv_para[5])

#                $(read(pipeline(`echo $(structure[i])`, `awk -v var=$(solv_para[5]) '{print $2, $3, $4, var, 1.2}'`), String)) $(solv_para[5]) 1.2
                

#geometry_new = read(pipeline(`echo $geometry`, `awk '{print $2, $3, $4}'`), String)

38

In [282]:
run(`cat tmp`)

%chk=file-i.chk
%nproc=8
%mem=1gb
#p wb97xd def2tzvp
scrf=(iefpcm,solvent=cyclohexane,read) nosym guess=only pop=none

title

0 1

C    1.10712900   -1.48465400    0.00000000
C   -0.00001100   -0.72877000    0.00000000
C    0.00000000    0.72875800    0.00000000
C   -1.10712300    1.48466300    0.00000000
H    2.09998900   -1.03986800    0.00000000
H    1.05995700   -2.56940600    0.00000000
H   -0.97811200   -1.21104800    0.00000000
H    0.97811000    1.21101800    0.00000000
H   -2.09999700    1.03991100    0.00000000
H   -1.05991600    2.56941400    0.00000000

qrep pcmdoc geomview nodis nocav g03defaults tsare=0.075
nsfe=10
eps=2.0165 rhos=0.7781 solvmw=84.1595


Process(`cat tmp`, ProcessExited(0))

In [203]:
structure_new = split(read(pipeline(`echo $geometry`, `awk '{print $2, $3, $4}'`), String), r"\n\s*\n+(?!$)")

3-element Array{SubString{String},1}:
 "1.10712900 -1.48465400 0.00000000\n-0.00001100 -0.72877000 0.00000000\n0.00000000 0.72875800 0.00000000\n-1.10712300 1.48466300 0.00000000\n2.09998900 -1.03986800 0.00000000\n1.05995700 -2.56940600 0.00000000\n-0.97811200 -1.21104800 0.00000000\n0.97811000 1.21101800 0.00000000\n-2.09999700 1.03991100 0.00000000\n-1.05991600 2.56941400 0.00000000"
 "1.10712900 -1.48465400 0.00000000\n-0.00001100 -0.72877000 0.00000000\n0.00000000 0.72875800 0.00000000\n-1.10712300 1.48466300 0.00000000\n2.09998900 -1.03986800 0.00000000\n1.05995700 -2.56940600 0.00000000\n-0.97811200 -1.21104800 0.00000000\n0.97811000 1.21101800 0.00000000\n-2.09999700 1.03991100 0.00000000\n-1.05991600 2.56941400 2.00000000"
 "1.10712900 -1.48465400 0.00000000\n-0.00001100 -0.72877000 0.00000000\n0.00000000 0.72875800 0.00000000\n-1.10712300 1.48466300 0.00000000\n2.09998900 -1.03986800 0.00000000\n1.05995700 -2.56940600 0.00000000\n-0.97811200 -1.21104800 0.00000000\n0.97811000